In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import joblib
import json
import numpy as np

sys.path.append('..')

In [2]:

data_dir = "/mnt/data/stephana93dm/storage/projects/alternative_clustering/alternative_clustering/datasets/"
dataset_name = "Fruit360"
load_dir = os.path.join(data_dir, dataset_name)
save_dir = os.path.join(data_dir, "processed", dataset_name)

img_dir = os.path.join(load_dir, "fruit360instance")

In [3]:
df = []
for fold in os.listdir(img_dir):
    fold_dir = os.path.join(img_dir, fold)
    if not os.path.isdir(fold_dir):
        continue
    assert len(os.listdir(fold_dir)) == 1
    for img in os.listdir(fold_dir):
        img_path = os.path.join(fold_dir, img)

        fruit = img.split("_")[0]
        color = img.split(" ")[0].split("_")[1]
        # df.append({"path": img_path, "fold": fold})
        df.append({"image_filename": img, "image_path": img_path, "fruit": fruit, "color": color})

df = pd.DataFrame(df)
df

,image_filename,image_path,fruit,color
0,cherry_yellow (329).jpg,/mnt/data/stephana93dm/storage/projects/altern...,cherry,yellow
1,banana_red (83).jpg,/mnt/data/stephana93dm/storage/projects/altern...,banana,red
2,banana_yellow (281).jpg,/mnt/data/stephana93dm/storage/projects/altern...,banana,yellow
3,apple_yellow (230).jpg,/mnt/data/stephana93dm/storage/projects/altern...,apple,yellow
4,cherry_red (694).jpg,/mnt/data/stephana93dm/storage/projects/altern...,cherry,red
...,...,...,...,...
4851,banana_yellow (182).jpg,/mnt/data/stephana93dm/storage/projects/altern...,banana,yellow
4852,cherry_burgundy (477).jpg,/mnt/data/stephana93dm/storage/projects/altern...,cherry,burgundy
4853,apple_green (117).jpg,/mnt/data/stephana93dm/storage/projects/altern...,apple,green
4854,grape_red (390).jpg,/mnt/data/stephana93dm/storage/projects/altern...,grape,red


In [4]:
from turtle import color
from sklearn.preprocessing import LabelEncoder

fruit_le = LabelEncoder()
color_le = LabelEncoder()

df["fruit_clusters"] = fruit_le.fit_transform(df["fruit"])
df["color_clusters"] = color_le.fit_transform(df["color"])

cluster_to_name = {
    "fruit": {i: name for i, name in enumerate(fruit_le.classes_)},
    "color": {i: name for i, name in enumerate(color_le.classes_)},
}

In [5]:
import shutil


save_folder = save_dir
image_save_folder = os.path.join(save_folder, "images")
clustering_save_folder = os.path.join(save_folder, "clustering")
os.makedirs(image_save_folder, exist_ok=True)
os.makedirs(clustering_save_folder, exist_ok=True)

with open(os.path.join(save_folder, "cluster_to_name.json"), "w") as f:
    json.dump(cluster_to_name, f)

for cluster_name in ["fruit", "color"]:
    dictionary = {}
    for idx, row in df.iterrows():
        name = row["image_filename"]
        dictionary[name] = row[f"{cluster_name}_clusters"]

        src = row["image_path"]
        dst = os.path.join(image_save_folder, name)
        shutil.copyfile(src, dst)
    
    save_file = os.path.join(clustering_save_folder, f"clusters_{cluster_name}.json")
    with open(save_file, "w") as f:
        json.dump(dictionary, f)